# EXPLORATION 9 : 영화 추천 시스템

## 준비한 데이터
- Movielens 데이터셋
- AlternatingLeastSquares 모델

## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 평점과 무비 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


## 2. 데이터 분석

In [8]:
# ratings에 있는 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# rating에 있는 사용자 수
ratings['user_id'].nunique()

6039

In [10]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3. 내가 선호하는 영화 추가

In [15]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('toy', regex=False)]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1948,2017,Babes in Toyland (1961),Children's|Fantasy|Musical
2184,2253,Toys (1992),Action|Comedy|Fantasy
2411,2480,Dry Cleaning (Nettoyage à sec) (1997),Drama
3017,3086,March of the Wooden Soldiers (a.k.a. Laurel & ...,Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy


In [16]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [2571, 1240, 589 , 1, 3114]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'Jinyoung'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['Jinyoung']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,Jinyoung,2571,5,"Matrix, The (1999)"
1,Jinyoung,1240,5,"Terminator, The (1984)"
2,Jinyoung,589,5,Terminator 2: Judgment Day (1991)
3,Jinyoung,1,5,Toy Story (1995)
4,Jinyoung,3114,5,Toy Story 2 (1999)


In [17]:
if not ratings.isin({'user_id':['Jinyoung']})['user_id'].any():  # user_id에 'Jinyoung'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,Jinyoung,2571,5,"Matrix, The (1999)"
836479,Jinyoung,1240,5,"Terminator, The (1984)"
836480,Jinyoung,589,5,Terminator 2: Judgment Day (1991)
836481,Jinyoung,1,5,Toy Story (1995)
836482,Jinyoung,3114,5,Toy Story 2 (1999)


## 4. indexing 후 CSR matrix 생성

In [18]:
# 고유한 유저와 영화의 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [19]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,2571,5,124
836479,6039,1240,5,200
836480,6039,589,5,92
836481,6039,1,5,40


In [20]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. AlternatingLeastSquares 모델 훈련시키기

In [21]:
from implicit.als import AlternatingLeastSquares
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분이며 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악

In [28]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
Jinyoung, matrix = user_to_idx['Jinyoung'], movie_to_idx['Matrix, The (1999)']
Jinyoung_vector, matrix_vector = als_model.user_factors[Jinyoung], als_model.item_factors[matrix]

In [29]:
Jinyoung_vector

array([ 0.10014746,  0.13090843,  1.2708162 , -0.68687576,  0.45094934,
        0.37077352,  0.9052938 ,  1.3788966 ,  0.07961525,  0.29669008,
       -0.75846195,  0.74388415, -0.47113392, -0.12932642, -0.93197787,
        0.14923303, -0.2406039 ,  0.08423883, -0.35523662, -0.32728055,
        0.54107237, -0.52516925, -0.19050725, -0.57920474, -1.442444  ,
        0.59177095, -0.41551685, -0.05796802,  0.23054636, -0.270266  ,
        0.20260563,  0.3571645 , -0.65003705,  0.05603706,  0.00653742,
        0.05656814, -0.31287342, -0.80896276, -0.5620988 , -0.14730056,
       -1.0344229 , -0.36246896,  0.7058259 , -0.33699158,  0.26209986,
        0.08202516,  0.04079275, -0.19201097, -0.6214019 ,  0.18658979,
       -0.08409049, -0.12562428, -0.06067752,  0.81054217,  0.06718295,
        0.20978726, -0.28485972,  0.7074986 , -0.18822263,  0.8648919 ,
        1.3863137 ,  0.37962756, -0.4905997 ,  0.30549353,  1.2255826 ,
        0.65006196,  0.17764845, -0.02682161, -0.44915017, -0.36

In [30]:
matrix_vector

array([ 0.00892773,  0.01395165,  0.03214344, -0.01907493,  0.03650445,
        0.01864339,  0.03210629,  0.03038034,  0.00918104, -0.00329341,
       -0.00790033,  0.04634393, -0.01027856,  0.01201372, -0.02446615,
        0.01231387,  0.01871398,  0.01356971,  0.01093933,  0.01022012,
        0.03063933, -0.00881045, -0.00208969,  0.00305655, -0.01544643,
        0.00115771, -0.01050997,  0.0029547 , -0.00994884,  0.01650683,
        0.00607065,  0.00642101, -0.02182644,  0.0171579 ,  0.00869644,
        0.00623343, -0.00577705, -0.0158711 , -0.00430821,  0.01200665,
       -0.04205878,  0.01325448,  0.01842709,  0.00607713,  0.01499997,
        0.0078981 ,  0.0127342 , -0.02249643, -0.00451698,  0.00142652,
        0.00957964, -0.0035417 ,  0.00014715,  0.01955912, -0.00845283,
        0.00504369,  0.00597893,  0.00715867,  0.01415879,  0.02521879,
        0.01094334,  0.01122033, -0.01367249, -0.00391551,  0.02414341,
        0.01937727,  0.00877264,  0.0325135 , -0.01011096, -0.00

In [32]:
# 나와 매트릭스의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(Jinyoung_vector, matrix_vector)

0.6545356

In [34]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['Titanic (1997)']
matrix_vector = als_model.item_factors[matrix]
np.dot(Jinyoung_vector, matrix_vector)

-0.008736022

## 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [35]:
favorite_movie = 'Terminator, The (1984)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(200, 0.99999994),
 (651, 0.78885543),
 (194, 0.7055871),
 (193, 0.69239587),
 (865, 0.66910094),
 (92, 0.62494993),
 (124, 0.5794355),
 (680, 0.5678494),
 (882, 0.54219496),
 (62, 0.5206272),
 (928, 0.51314753),
 (172, 0.47939712),
 (117, 0.47907722),
 (120, 0.4737691),
 (195, 0.4636106)]

In [36]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Terminator, The (1984)',
 'Aliens (1986)',
 'Die Hard (1988)',
 'Alien (1979)',
 'Predator (1987)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Blade Runner (1982)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Total Recall (1990)',
 'Robocop (1987)',
 'Indiana Jones and the Last Crusade (1989)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (1981)',
 'Mad Max (1979)']

In [37]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [38]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Aladdin (1992)',
 'Babe (1995)',
 "Bug's Life, A (1998)",
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)',
 "There's Something About Mary (1998)"]

## 8. 내가 가장 좋아할 만한 영화 추천

In [39]:
user = user_to_idx['Jinyoung']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(62, 0.396303),
 (141, 0.38565862),
 (651, 0.33436698),
 (4, 0.32172668),
 (107, 0.29286158),
 (38, 0.28846142),
 (193, 0.288359),
 (22, 0.28685683),
 (110, 0.26926446),
 (882, 0.2620968),
 (33, 0.2612274),
 (317, 0.25687808),
 (194, 0.25135386),
 (322, 0.24136728),
 (680, 0.23531826),
 (160, 0.23019478),
 (121, 0.22677606),
 (44, 0.22341746),
 (117, 0.22303915),
 (75, 0.2195437)]

In [40]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Total Recall (1990)',
 'Fugitive, The (1993)',
 'Aliens (1986)',
 "Bug's Life, A (1998)",
 'Jurassic Park (1993)',
 'Sixth Sense, The (1999)',
 'Alien (1979)',
 'Back to the Future (1985)',
 'Groundhog Day (1993)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Aladdin (1992)',
 'Twelve Monkeys (1995)',
 'Die Hard (1988)',
 'Babe (1995)',
 'Blade Runner (1982)',
 'Forrest Gump (1994)',
 'Silence of the Lambs, The (1991)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Hunt for Red October, The (1990)']

In [41]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Total Recall (1990)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Terminator 2: Judgment Day (1991)', 0.18374904017632054),
 ('Matrix, The (1999)', 0.13173966515357244),
 ('Terminator, The (1984)', 0.08551039715602318),
 ('Toy Story (1995)', -0.0004882841138337368),
 ('Toy Story 2 (1999)', -0.009545283728951822)]

# 프로젝트 결과

#### 데이터 전처리 
ratings.dat와 movies.dat 파일을 어차피 모두 사용해야하기 때문에 movie_id를 기준으로 join하고 시작했다.  

#### 결과  
1. 영화 갯수 : 3628 , 사용자 수 : 6039에 내가 선호하는 데이터를 더 추가하여 CSR matrix를 통해 사용자수는 6040으로 늘어난 것을 볼 수 있다.
2. AlternatingLeastSquares 모델 훈련을 통해 사용자 벡터와 비교할 벡터의 내적값으로 유사도를 확인하였다.
3. similar_items 메서드, recommend 메서드, explain 메소드를 이용하여 영화 추천과 기여도를 확인하였다.

#### 회고
1. 처음에 어떤 영화를 내가 좋아하는 영화로 추가할지를 검색할때 소문자와 대문자로 영화 제목이 섞여 있으니 전부 소문자로 만들어주고 검색하는 아이디어가 인상깊었다.
2. 유튜브, 넷플릭스, 멜론 등에서 사용하는 추천시스템이 어떤식으로 코딩되어있는지 감을 잡을 수 있어서 좋았다.
3. 협업 필터링과 콘텐츠 기반 필터링 방식의 차이가 협업 필터링은 다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악하지만, 콘텐츠 기반 필터링은 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악하는 필터링 방식의 차이를 알게되어 좋았다.
4. 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋 중 좋아요나 별점처럼 선호도를 명시적(explicit)으로 나타내는 것과 서비스를 사용하면서 자연스럽게 발생하는 암묵적(implicit)인 피드백을 이용할 수 있다는 것을 알게되어 좋았다.
5. 암묵적인 피드백 중 방문 페이지 이력과 검색 기록, 심지어는 마우스 움직임 기록까지 이용된다는데 실제로 이런 암묵적인 피드백을 어떻게 코드로 구성해서 이용하는건지 아직 감이 안온다. 개발자들은 정말 대단한 것 같다.